In [1]:
#Pandas is a tool used to help us process dataframes/tables, which are csv-like
#The Natural Language Toolkit is the package that helps us in the lemmatization and part-of-speech tagging

import pandas as pd
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet
import nltk

In [2]:
#NOTE: the nltk.download() was giving me trouble so I used: import nltk
#Then: nltk.download('wordnet') and nltk.download('wordnet_ic')
#To get the specific data packs we need

#testing if it worked...

#Another note: lemmatizer and stemmer seem not to be doing what I hoped
#Might need to use VerbNet? Look into this next...
wnl = WordNetLemmatizer()
ps = PorterStemmer()
wnl.lemmatize('dogs')
ps.stem('is')
text = ["Rylund","this"]
nltk.pos_tag(["this"])

[('this', 'DT')]

In [3]:
#This is a helper function that allows us to split up the text and make it uniform for processing
def cleaner(word):
    #This "if" line removes "@" from the end of words that have it
    if "@" in word:
        loc = word.find("@")
        word = word[:loc]

    return word.replace(",","").replace("(","").replace("\"","").replace(")","").strip(r" .?![?]").split()

#Helper funciton to map part-of-speech tags to something that our root-finding map "wordnet" can recognize
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def tag_helper(word,tag):
    wntag = get_wordnet_pos(tag)
    if wntag is None:# not supply tag in case of None
        lemma = wnl.lemmatize(word)
        tb_tag = ""
    #assigning a variable to the english equivalent of our tags to be used later when we build the csv
    else:
        lemma = wnl.lemmatize(word, pos = wntag)
        if wntag == wordnet.ADJ:
            tb_tag = "Adjective"
        elif wntag == wordnet.VERB:
            tb_tag = "Verb"
        elif wntag == wordnet.NOUN:
            tb_tag = "Noun"
        elif wntag == wordnet.ADV:
            tb_tag = "Adverb"
        elif wntag is None:
            tb_tag = ""
    return lemma, tb_tag

def save_xls(list_dfs, xls_path):
    writer = pd.ExcelWriter(xls_path)
    for name, df in list_dfs:
        df.to_excel(writer,'%s' % name)
    writer.save()

In [4]:
#Here we load in the data
xls = pd.ExcelFile("Brent Corpus Samples_10_08_17.xlsx")
vocab_xls = pd.ExcelFile("Core Vocabulary_(Preschool).xlsx")
word_map_xlsx = pd.ExcelFile("word_map.xlsx")

#sheet_names stores all the names of the sheets from the excel file
sheet_names = xls.sheet_names
vocab_sheet_names = vocab_xls.sheet_names

sheets = []
for name in sheet_names:
    if not name[-1].isalpha() and name != "Sheet102":
        sheets.append((name,xls.parse(name)))

#Note the spelling error in "36 Univeresal Core"... Entered in the CSV that way
core_sheets = [("36 Core", xls.parse('36 Univeresal Core')),
              ("100 Core", xls.parse('100 Universal Core')),
              ("All Core", xls.parse("All Universal Core"))]

preschool_sheets = []
for name in vocab_sheet_names:
    if name != "All Three":
        preschool_sheets.append((name, vocab_xls.parse(name)))

word_map_sheet = ("Word Map", word_map_xlsx.parse('word_map'))
        
core_36_list = [word.lower() for word in core_sheets[0][1]["Words"]]
core_100_list = [word.lower() for word in core_sheets[1][1]["Words"]]
core_all_list = [word.lower() for word in core_sheets[2][1]["Words"]]

banajee_list = [word.lower() for word in preschool_sheets[0][1]["Vocabulary"]]
beukelman_list = [word.lower() for word in preschool_sheets[1][1]["Vocabulary"]]
marvin_list = [word.lower() for word in preschool_sheets[2][1]["Vocabulary"]]

word_map = {key: value for key,value in zip(word_map_sheet[1]["Lemmed Words"], word_map_sheet[1]["Map"])}

In [7]:
total_corpus = []
sheet_corpus = {}
raw_word_count_by_sheet = {}
lem_word_count_by_sheet = {}
total_raw_word_count = {}
total_lem_word_count = {}
tagged_sheet = {}
word_mapped = {}
raw_mother_dict = {}
raw_count_by_mother = {}
lem_mother_dict = {}
total_lemmed = []
commonality = {}

for name,sheet in sheets:
    #print(name)
    code = sheet['@Begin']
    text = sheet['Unnamed: 1']
    temp_raw_word_count = {}
    temp_lem_word_count = {}
    temp_corpus = []
    abbrev = name[:2].lower()
    if abbrev not in raw_count_by_mother:
        raw_count_by_mother[abbrev] = {}
    
    for c,t in zip(code,text):
        if c == "*MOT:":
            
            for word in cleaner(t):
                if word[0].isalpha() and word[-1].isalpha(): #need to change this so that those words aren't just...
                                                         #...thrown out.
                    word = word.lower()
                    #Making a big list of all the words used called "corpus"
                    total_corpus.append(word)
                    temp_corpus.append(word)
                    #word_count counts the words for us
                    if word not in temp_raw_word_count:
                        temp_raw_word_count[word] = [0,0,0,0,0,0,0]
                    temp_raw_word_count[word][0] = temp_raw_word_count.get(word)[0] + 1
                    if word not in total_raw_word_count:
                        total_raw_word_count[word] = [0,0,0,0,0,0,0]
                    total_raw_word_count[word][0] = total_raw_word_count.get(word)[0] + 1
                    if word not in raw_count_by_mother[abbrev]:
                        raw_count_by_mother[abbrev][word] = [0,0,0,0,0,0,0]
                    raw_count_by_mother[abbrev][word][0] = raw_count_by_mother[abbrev].get(word)[0] + 1
                    
                    if word in core_36_list:
                        total_raw_word_count[word][1] = 1
                    
                    if word in core_100_list:
                        total_raw_word_count[word][2] = 1
                    
                    if word in core_all_list:
                        total_raw_word_count[word][3] = 1
                        
                    if word in banajee_list:
                        total_raw_word_count[word][4] = 1

                    if word in beukelman_list:
                        total_raw_word_count[word][5] = 1

                    if word in marvin_list:
                        total_raw_word_count[word][6] = 1
                    
                    for i in range(1,7):
                        temp_raw_word_count[word][i] = total_raw_word_count[word][i]
                        raw_count_by_mother[abbrev][word][i] = total_raw_word_count[word][i]
                    
                    
    raw_word_count_by_sheet[name] = temp_raw_word_count
    lem_word_count_by_sheet[name] = temp_lem_word_count
    sheet_corpus[name] = temp_corpus
    tagged_sheet[name] = nltk.pos_tag(temp_corpus)

In [231]:
total_lem_word_count = {}
lem_word_count_by_mother = {}
lem_count_by_sheet = {}
for sheet, words in tagged_sheet.items():
    abbrev = sheet[:2].lower()
    if abbrev not in lem_word_count_by_mother:
        lem_word_count_by_mother[abbrev] = {}
    lem_count_by_sheet[sheet] = {}
    
    for word in words:
        word,tag = word
        lemma,tb_tag = tag_helper(word,tag)
        if lemma in word_map:
            lemma = word_map[lemma]
        
        if lemma not in total_lem_word_count:
            total_lem_word_count[lemma] = [0,0,0,0,0,0,0]
        total_lem_word_count[lemma][0] = total_lem_word_count.get(lemma)[0] + 1
        
        if lemma not in lem_word_count_by_mother[abbrev]:
            lem_word_count_by_mother[abbrev][lemma] = [0,0,0,0,0,0,0]
        lem_word_count_by_mother[abbrev][lemma][0] = lem_word_count_by_mother[abbrev].get(lemma)[0] + 1
        
        if lemma not in lem_count_by_sheet[sheet]:
            lem_count_by_sheet[sheet][lemma] = [0,0,0,0,0,0,0]
        lem_count_by_sheet[sheet][lemma][0] = lem_count_by_sheet[sheet].get(lemma)[0] + 1


        if lemma in core_36_list:
            total_lem_word_count[lemma][1] = 1

        if lemma in core_100_list:
            total_lem_word_count[lemma][2] = 1

        if lemma in core_all_list:
            total_lem_word_count[lemma][3] = 1

        if lemma in banajee_list:
            total_lem_word_count[lemma][4] = 1

        if lemma in beukelman_list:
            total_lem_word_count[lemma][5] = 1

        if lemma in marvin_list:
            total_lem_word_count[lemma][6] = 1
                        
        for i in range(1,7):
            lem_word_count_by_mother[abbrev][lemma][i] = total_lem_word_count[lemma][i]
            lem_count_by_sheet[sheet][lemma][i] = total_lem_word_count[lemma][i]

In [232]:
lem_count = sum([x[0] for x in total_lem_word_count.values()])
total_lem = pd.Series([x[0] for x in total_lem_word_count.items()])
total_lem_rel_freq = pd.Series([(x[1][0]/lem_count)*100 
                          for x in total_lem_word_count.items()])
total_lem_count = pd.Series([x[1][0] for x in total_lem_word_count.items()])
total_lem_core_36 = pd.Series([x[1][1] for x in total_lem_word_count.items()])
total_lem_core_100 = pd.Series([x[1][2] for x in total_lem_word_count.items()])
total_lem_core_all = pd.Series([x[1][3] for x in total_lem_word_count.items()])
total_lem_banajee = pd.Series([x[1][4] for x in total_lem_word_count.items()])
total_lem_beukelman = pd.Series([x[1][5] for x in total_lem_word_count.items()])
total_lem_marvin = pd.Series([x[1][6] for x in total_lem_word_count.items()])

lem_total_df = pd.DataFrame({"Lemmed Words": total_lem, "Count": total_lem_count,
                            "Relative Freq": total_lem_rel_freq, "Core 36": total_lem_core_36,
                            "Core 100": total_lem_core_100, "Core All": total_lem_core_all,
                            "Banajee et al": total_lem_banajee, "Beukelman et al": total_lem_beukelman,
                            "Marvin et al": total_lem_marvin},
                           columns = ["Lemmed Words", "Count", "Relative Freq",
                                     "Core 36", "Core 100", "Core All", "Banajee et al",
                                     "Beukelman et al", "Marvin et al"])
lem_total_df.reset_index().drop("index", axis = 1).sort_values("Relative Freq")
lem_total_df.to_excel("Lem Corpus 11_5.xlsx")

In [233]:
lem_mother_dfs = []
for mother,value in lem_word_count_by_mother.items():
    current = value
    lems = pd.Series([x[0] for x in current.items()])
    lem_count = sum([x[0] for x in current.values()])
    lem_rel_freq = pd.Series([(x[1][0]/lem_count)*100 
                          for x in current.items()])
    lem_count = pd.Series([x[1][0] for x in current.items()])
    lem_core_36 = pd.Series([x[1][1] for x in current.items()])
    lem_core_100 = pd.Series([x[1][2] for x in current.items()])
    lem_core_all = pd.Series([x[1][3] for x in current.items()])
    lem_banajee = pd.Series([x[1][4] for x in current.items()])
    lem_beukelman = pd.Series([x[1][5] for x in current.items()])
    lem_marvin = pd.Series([x[1][6] for x in current.items()])
    temp_df = pd.DataFrame({"Lemmed Words": lems, "Count": lem_count,
                            "Relative Freq": lem_rel_freq, "Core 36": lem_core_36,
                            "Core 100": lem_core_100, "Core All": lem_core_all,
                            "Banajee et al": lem_banajee, "Beukelman et al": lem_beukelman,
                            "Marvin et al": lem_marvin},
                           columns = ["Lemmed Words", "Count", "Relative Freq",
                                     "Core 36", "Core 100", "Core All", "Banajee et al",
                                     "Beukelman et al", "Marvin et al"])
    temp_df.reset_index().drop("index", axis = 1).sort_values("Relative Freq")
    lem_mother_dfs.append((mother,temp_df))

save_xls(lem_mother_dfs, "Lem by Mother 11_5.xlsx")

In [234]:
lem_sheet_dfs = []
for sheet,value in lem_count_by_sheet.items():
    current = value
    lems = pd.Series([x[0] for x in current.items()])
    lem_count = sum([x[0] for x in current.values()])
    lem_rel_freq = pd.Series([(x[1][0]/lem_count)*100 
                          for x in current.items()])
    lem_count = pd.Series([x[1][0] for x in current.items()])
    lem_core_36 = pd.Series([x[1][1] for x in current.items()])
    lem_core_100 = pd.Series([x[1][2] for x in current.items()])
    lem_core_all = pd.Series([x[1][3] for x in current.items()])
    lem_banajee = pd.Series([x[1][4] for x in current.items()])
    lem_beukelman = pd.Series([x[1][5] for x in current.items()])
    lem_marvin = pd.Series([x[1][6] for x in current.items()])
    temp_df = pd.DataFrame({"Lemmed Words": lems, "Count": lem_count,
                            "Relative Freq": lem_rel_freq, "Core 36": lem_core_36,
                            "Core 100": lem_core_100, "Core All": lem_core_all,
                            "Banajee et al": lem_banajee, "Beukelman et al": lem_beukelman,
                            "Marvin et al": lem_marvin},
                           columns = ["Lemmed Words", "Count", "Relative Freq",
                                     "Core 36", "Core 100", "Core All", "Banajee et al",
                                     "Beukelman et al", "Marvin et al"])
    temp_df.reset_index().drop("index", axis = 1).sort_values("Relative Freq")
    lem_sheet_dfs.append((sheet,temp_df))

save_xls(lem_sheet_dfs, "Lem by Sessions 11_5.xlsx")

In [223]:
raw_count = sum([x[0] for x in total_raw_word_count.values()])
total_raw = pd.Series([x[0] for x in total_raw_word_count.items()])
total_raw_count = pd.Series([x[1][0] for x in total_raw_word_count.items()])
raw_rel_freq = pd.Series([(x[1][0]/raw_count)*100 
                          for x  in total_raw_word_count.items()])
raw_core_36 = pd.Series([x[1][1] for x in total_raw_word_count.items()])
raw_core_100 = pd.Series([x[1][2] for x in total_raw_word_count.items()])
raw_core_all = pd.Series([x[1][3] for x in total_raw_word_count.items()])
raw_banajee = pd.Series([x[1][4] for x in total_raw_word_count.items()])
raw_beukelman = pd.Series([x[1][5] for x in total_raw_word_count.items()])
raw_marvin = pd.Series([x[1][6] for x in total_raw_word_count.items()])

raw_total_df = pd.DataFrame({"Raw Words": total_raw, "Count": total_raw_count,
                            "Relative Freq": raw_rel_freq, "Core 36": raw_core_36,
                            "Core 100": raw_core_100, "Core All": raw_core_all,
                            "Banajee et al": raw_banajee, "Beukelman et al": raw_beukelman,
                            "Marvin et al": raw_marvin},
                           columns = ["Raw Words", "Count", "Relative Freq",
                                     "Core 36", "Core 100", "Core All",
                                     "Banajee et al", "Beukelman et al", "Marvin et al"])
raw_total_df.reset_index().drop("index", axis = 1).sort_values("Relative Freq")
raw_total_df.to_excel("Raw Corpus 11_5.xlsx")
print("no more of that...")

no more of that...


In [39]:
commonality = {}
for mother, values in raw_count_by_mother.items():
    for word, count in values.items():
        if word not in commonality:
            commonality[word] = 0
        commonality[word] = commonality[word] + 1
commonality_word = pd.Series([x[0] for x in commonality.items()])
commonality_count = pd.Series([x for x in commonality.values()])
common_df = pd.DataFrame({"Words": commonality_word, "Counts": commonality_count},
                        columns = ["Words", "Count"])
#common_df = pd.DataFrame.from_dict(commonality, orient = "index")
#common_df.to_excel("Commonality Across Mothers.xlsx")

new_raw_count_by_mother = {abbrev[0]: {} for abbrev in raw_count_by_mother.items()}
for mom, dic in raw_count_by_mother.items():
    for word, count in dic.items():
        new_raw_count_by_mother[mom][word] = [count[0], count[1], count[2], count[3],
                                                 count[4], count[5], count[6], commonality[word]]

In [42]:
raw_mother_dfs = []
#changing to new_raw_count_by_mother
for mother,value in new_raw_count_by_mother.items():
    current = value
    raws = pd.Series([x[0] for x in current.items()])
    raw_count = sum([x[0] for x in current.values()])
    raw_rel_freq = pd.Series([(x[1][0]/raw_count)*100 
                          for x in current.items()])
    raw_count = pd.Series([x[1][0] for x in current.items()])
    raw_core_36 = pd.Series([x[1][1] for x in current.items()])
    raw_core_100 = pd.Series([x[1][2] for x in current.items()])
    raw_core_all = pd.Series([x[1][3] for x in current.items()])
    raw_banajee = pd.Series([x[1][4] for x in current.items()])
    raw_beukelman = pd.Series([x[1][5] for x in current.items()])
    raw_marvin = pd.Series([x[1][6] for x in current.items()])
    common = pd.Series([x[1][7] for x in current.items()])
    temp_df = pd.DataFrame({"Raw Words": raws, "Count": raw_count,
                            "Relative Freq": raw_rel_freq, "Core 36": raw_core_36,
                            "Core 100": raw_core_100, "Core All": raw_core_all,
                            "Banajee et al": raw_banajee, "Beukelman et al": raw_beukelman,
                            "Marvin et al": raw_marvin, "Commonality": common},
                           columns = ["Raw Words", "Count", "Relative Freq",
                                     "Core 36", "Core 100", "Core All", "Banajee et al",
                                     "Beukelman et al", "Marvin et al", "Commonality"])
    raw_mother_dfs.append((mother,temp_df))
    


save_xls(raw_mother_dfs, "Raw by Mother 12_4.xlsx")


In [225]:
raw_sheet_dfs = []
for session,value in raw_word_count_by_sheet.items():
    current = value
    raws = pd.Series([x[0] for x in current.items()])
    raw_count = sum([x[0] for x in current.values()])
    raw_rel_freq = pd.Series([(x[1][0]/raw_count)*100 
                          for x in current.items()])
    raw_count = pd.Series([x[1][0] for x in current.items()])
    raw_core_36 = pd.Series([x[1][1] for x in current.items()])
    raw_core_100 = pd.Series([x[1][2] for x in current.items()])
    raw_core_all = pd.Series([x[1][3] for x in current.items()])
    raw_banajee = pd.Series([x[1][4] for x in current.items()])
    raw_beukelman = pd.Series([x[1][5] for x in current.items()])
    raw_marvin = pd.Series([x[1][6] for x in current.items()])
    temp_df = pd.DataFrame({"Raw Words": raws, "Count": raw_count,
                            "Relative Freq": raw_rel_freq, "Core 36": raw_core_36,
                            "Core 100": raw_core_100, "Core All": raw_core_all,
                            "Banajee et al": raw_banajee, "Beukelman et al": raw_beukelman,
                            "Marvin et al": raw_marvin},
                           columns = ["Raw Words", "Count", "Relative Freq",
                                     "Core 36", "Core 100", "Core All", "Banajee et al",
                                     "Beukelman et al", "Marvin et al"])
    raw_sheet_dfs.append((session,temp_df))

save_xls(raw_sheet_dfs, "Raw by Session 11_5.xlsx")

In [235]:
counter = 0
for key,value in lem_word_count_by_mother.items():
    for word,count in value.items():
            counter += count[0]
print(counter)
new_count = 0
for key,value in total_lem_word_count.items():
    new_count += value[0]
print(new_count)
new_new = 0
for key,value in lem_count_by_sheet.items():
    for word,count in value.items():
        new_new += count[0]
print(new_new)

267204
267204
267204


In [229]:
new_new = 0
print(len(total_corpus))
counter = 0
for count in total_raw_word_count.values():
    counter += count[0]
print(counter)
new_counter = 0
for key,value in raw_count_by_mother.items():
    for word,count in value.items():
        new_counter += count[0]
print(new_counter)
new_new_counter = 0
for key,value in raw_word_count_by_sheet.items():
    for word,count in value.items():
        new_new_counter += count[0]
print(new_new_counter)

267204
267204
267204
267204


In [36]:
#Note total_lem_word_count's list for value goes like this:
# [count, 36 yes/no, 100 yes/no, all yes/no] for universal core binary
total_lem_word_count = {}
total_lemmed = []

for word, tag in total_tagged:
    
    lemma, tb_tag = tag_helper(word,tag)
    
    if lemma in word_map:
        lemma = word_map[lemma]
    
    #builing out lemmed here
    total_lemmed.append((word,lemma,tb_tag))
    #builing out the count here
    if lemma not in total_lem_word_count:
        total_lem_word_count[lemma] = [0,0,0,0,0,0,0]
    total_lem_word_count[lemma][0] = total_lem_word_count.get(lemma)[0] + 1
    
    if lemma in core_36_list:
        total_lem_word_count[lemma][1] = 1
    
    if lemma in core_100_list:
        total_lem_word_count[lemma][2] = 1
    
    if lemma in core_all_list:
        total_lem_word_count[lemma][3] = 1
        
    if lemma in banajee_list:
        total_lem_word_count[lemma][4] = 1
        
    if lemma in beukelman_list:
        total_lem_word_count[lemma][5] = 1
        
    if lemma in marvin_list:
        total_lem_word_count[lemma][6] = 1


lem_count = sum([x[0] for x in total_lem_word_count.values()])
total_lem = pd.Series([x[0] for x in total_lem_word_count.items()])
lem_rel_freq = pd.Series([(x[1][0]/lem_count)*100 
                          for x in total_lem_word_count.items()])
total_lem_count = pd.Series([x[1][0] for x in total_lem_word_count.items()])
lem_core_36 = pd.Series([x[1][1] for x in total_lem_word_count.items()])
lem_core_100 = pd.Series([x[1][2] for x in total_lem_word_count.items()])
lem_core_all = pd.Series([x[1][3] for x in total_lem_word_count.items()])
lem_banajee = pd.Series([x[1][4] for x in total_lem_word_count.items()])
lem_beukelman = pd.Series([x[1][5] for x in total_lem_word_count.items()])
lem_marvin = pd.Series([x[1][6] for x in total_lem_word_count.items()])

lem_total_df = pd.DataFrame({"Lemmed Words": total_lem, "Count": total_lem_count,
                            "Relative Freq": lem_rel_freq, "Core 36": lem_core_36,
                            "Core 100": lem_core_100, "Core All": lem_core_all,
                            "Banajee et al": lem_banajee, "Beukelman et al": lem_beukelman,
                            "Marvin et al": lem_marvin},
                           columns = ["Lemmed Words", "Count", "Relative Freq",
                                     "Core 36", "Core 100", "Core All", "Banajee et al",
                                     "Beukelman et al", "Marvin et al"])
lem_total_df.reset_index().drop("index", axis = 1).sort_values("Relative Freq")

lem_total_df.to_csv("Corpus - Lemmed words 11_5.csv")


raw_count = sum([x[0] for x in total_raw_word_count.values()])
total_raw = pd.Series([x[0] for x in total_raw_word_count.items()])
total_raw_count = pd.Series([x[1][0] for x in total_raw_word_count.items()])
raw_rel_freq = pd.Series([(x[1][0]/raw_count)*100 
                          for x  in total_raw_word_count.items()])
raw_core_36 = pd.Series([x[1][1] for x in total_raw_word_count.items()])
raw_core_100 = pd.Series([x[1][2] for x in total_raw_word_count.items()])
raw_core_all = pd.Series([x[1][3] for x in total_raw_word_count.items()])
raw_banajee = pd.Series([x[1][4] for x in total_raw_word_count.items()])
raw_beukelman = pd.Series([x[1][5] for x in total_raw_word_count.items()])
raw_marvin = pd.Series([x[1][6] for x in total_raw_word_count.items()])

raw_total_df = pd.DataFrame({"Raw Words": total_raw, "Count": total_raw_count,
                            "Relative Freq": raw_rel_freq, "Core 36": raw_core_36,
                            "Core 100": raw_core_100, "Core All": raw_core_all,
                            "Banajee et al": raw_banajee, "Beukelman et al": raw_beukelman,
                            "Marvin et al": raw_marvin},
                           columns = ["Raw Words", "Count", "Relative Freq",
                                     "Core 36", "Core 100", "Core All",
                                     "Banajee et al", "Beukelman et al", "Marvin et al"])
raw_total_df.reset_index().drop("index", axis = 1).sort_values("Relative Freq")

raw_total_df.to_csv("Corpus - Raw words 11_5.csv")